# Проект "Сервис для предсказания стоимости домов на основе истории предложений"

"Агентство недвижимости" 

К нам обратился представитель крупного агентства недвижимости со
следующей проблемой:
«Мои риелторы тратят катастрофически много времени на сортировку
объявлений и поиск выгодных предложений. Поэтому их скорость реакции, да
и, сказать по правде, качество анализа не дотягивают до уровня конкурентов.
Это сказывается на наших финансовых показателях.

Наша задача — разработать модель, которая позволила бы обойти
конкурентов по скорости и качеству совершения сделок. Вот датасет для
работы».

Что необходимо сделать: разработать сервис для предсказания стоимости
домов на основе истории предложений.

In [ ]:
# Импорт библиотек
import os
import sys
import warnings

# ==========================================
# 1. CPU ACCELERATION (Intel)
# ==========================================
try:
    from sklearnex import patch_sklearn
    patch_sklearn()
    print('Intel Extension (CPU) активирован для sklearn')
except (ImportError, Exception):
    print('Intel Extension не найден, работаем на стандартном CPU')

# ==========================================
# 2. GPU ACCELERATION (NVIDIA RAPIDS)
# ==========================================
USE_GPU = False

# Попытка включить "Магический Pandas" (работает только в Jupyter)
try:
    from IPython.core.getipython import get_ipython
    if get_ipython() is not None:
        get_ipython().run_line_magic('load_ext', 'cudf.pandas')
        print("Magic Mode: Pandas автоматически ускорен на GPU!")
        USE_GPU = True
except (ImportError, ModuleNotFoundError):
    pass # Мы не в Jupyter или нет cudf.pandas

import pandas as pd
import numpy as np

# Если магия не сработала (мы в скрипте .py), пробуем импортировать cudf явно
if not USE_GPU:
    try:
        import cudf
        print("Explicit Mode: Библиотека cuDF доступна (используйте cudf вместо pd)")
        USE_GPU = True
    except ImportError:
        print("GPU не найден, работаем на чистом CPU")

# Импорт ML-библиотек (cuml)
try:
    import cuml
    print("✅ cuML (GPU Machine Learning) доступен")
except ImportError:
    pass

from IPython.display import display, Markdown
from matplotlib import pyplot as plt
from tqdm import tqdm

if USE_GPU:
    # Импортируем GPU-версии
    from cuml.ensemble import RandomForestClassifier
    from cuml.linear_model import LinearRegression, LogisticRegression
    from cuml.neighbors import KNeighborsClassifier
    from cuml.svm import SVC
    # Можно добавить любые другие модели, которые есть в cuML    
    print("Модели: загружены из cuML (GPU)")    
else:
    # Импортируем стандартные CPU-версии
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LinearRegression, LogisticRegression
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.svm import SVC    
    print("Модели: загружены из Scikit-learn (CPU)")


from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

#from arch import arch_model
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты
from surprise.model_selection import train_test_split
from surprise import SVD, KNNBasic, accuracy
from surprise import BaselineOnly

from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
from scipy.linalg import svd
from scipy.optimize import minimize, least_squares

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

import matplotlib.ticker as ticker
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = 'browser'
from plotly.subplots import make_subplots


import ast
import datetime

from dmba import AIC_score, stepwise_selection
from catboost import CatBoostRegressor
import xgboost as xgb
import optuna
import pickle
import joblib

import scipy
import math

import time


Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


Intel Extension (CPU) активирован для sklearn
Magic Mode: Pandas автоматически ускорен на GPU!
✅ cuML (GPU Machine Learning) доступен


ModuleNotFoundError: No module named 'tqdm'

In [ ]:
filterwarnings("ignore")
plt.style.use('seaborn-v0_8') #стиль отрисовки seaborn
sns.set_style("whitegrid")

SEED = 42

In [ ]:
data = pd.read_csv('./data/data.csv')
data.head()